Libraries that I need to install

In [1]:
!pip install bert-for-tf2
!pip install sentencepiece
# !pip install pdpipe 
# !pip install symspellpy
!pip install pycontractions

     |████████████████████████████████| 40 kB 1.7 MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.13.5-py3-none-any.whl size=29946 sha256=8ac17447396905ca12ef90886d34726ecc6a03b781c1db2486c809e765088a40
  Stored in directory: /root/.cache/pip/wheels/90/00/4a/e3ae98003e155850dee523617ced7794e6e3da07d27e3802a0
  Created wheel for py-params: filename=py_params-0.9.6-py3-none-any.whl size=7088 sha256=05806a4c7d742384c0756263ce171c43c30a183fbb409bd31570afb2f4ee005b
  Stored in directory: /root/.cache/pip/wheels/d1/4d/5a/7fbbdfb87bf3da12265e0d79dc19ce143652a09e3696189eb8
  Created wheel for params-flow: filename=params_flow-0.7.4-py3-none-any.whl size=16194 sha256=9e8bf7a360f2a6248771e391651acd86a0ec0ca9c2a2c026a2f8118c5a3289e7
  Stored in directory: /root/.cache/pip/wheels/f4/90/ca/a5d500c088762773489e532acd0268226cf2597f211928f38a
Successfully built bert-for-tf2 py-params params-flow
  Created wheel for language-check: filename=language_check-1.1-py3-none-any.whl size=90190

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam



########################################################
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub



try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert


#data pipeline 
# import pdpipe as pdp

#Other Preprocessing
import re
# from sklearn.pipeline import Pipeline
# from sklearn.pipeline import FeatureUnion
import string
# !pip install symspellpy
import pkg_resources
# from symspellpy.symspellpy import SymSpell
# from symspellpy import SymSpell, Verbosity
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import csv
import pandas as pd

import nltk
nltk.download('punkt')

#Contraction Import
from pycontractions import Contractions

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/train.csv


Using TensorFlow backend.


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Reading the text and storing it... 

In [3]:
train= pd.read_csv('../input/nlp-getting-started/train.csv')
test=pd.read_csv('../input/nlp-getting-started/test.csv')
train.head(3)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1


## Web Scrapping done for Abbreviations and then preprocessed
Version Edit: Anit G.
* Removed Abusive words
* Removed empty slangs
* Removed slangs such as "?- it means some body is asking question"
* Removed slangs with length more than 7

In [4]:
# from bs4 import BeautifulSoup
# import requests, json, csv
# import re

# def remove_wrong_abb(key,value):
#     if(re.search('it means',value)):
#         return False;
#     elif(re.search('\*',value)):
#         return False
#     elif(value==""):
#         return False
#     elif(len(key)>7):
#         return False
#     else :
#         return True
    
# resp = requests.get("http://www.netlingo.com/acronyms.php")
# soup = BeautifulSoup(resp.text, "html.parser")
# slangdict= {}
# key=""
# value=""
# for div in soup.findAll('div', attrs={'class':'list_box3'}):
#     for li in div.findAll('li'):
#         for a in li.findAll('a'):
#             key =a.text
#             value = li.text.split(key)[1]
#             if(remove_wrong_abb(key,value)):
#                 if(re.search('\ -or- ',value)):
#                     pos=re.search('\ -or- ',value).start()
#                     slangdict[key]=value[:pos]
#                 else:
#                     slangdict[key]=value

                    
# # with open('myslang.json', 'w') as f:
# #     json.dump(slangdict, f, indent=2)

# w = csv.writer(open("myslang.csv", "w"))
# for key, val in slangdict.items():
#     w.writerow([key.lower(), val.lower()])
    
# reader = csv.reader(open('myslang.csv', 'r'))
# abbreviations = {}
# for row in reader:
#     k, v = row
#     abbreviations[k] = v


## All Preprocessing Configuration and Methods 

## SymSpell - Unbeatable Spell Checker

**Configuration Setup**

In [5]:
# #Version Edit: Sonam D.
# sym_spell_4space = SymSpell(max_dictionary_edit_distance=0, prefix_length=7)
# dictionary_path = pkg_resources.resource_filename(
#     "symspellpy", "frequency_dictionary_en_82_765.txt")
# sym_spell_4space.load_dictionary(dictionary_path, term_index=0, count_index=1)


# sym_spell_misspelled = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
# bigram_path = pkg_resources.resource_filename(
#     "symspellpy", "frequency_bigramdictionary_en_243_342.txt")
# sym_spell_misspelled.load_dictionary(dictionary_path, term_index=0, count_index=1)
# sym_spell_misspelled.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

## Google Trained for Contraction handling - pycontractions

**Configuration Setup**
[](http://)

In [6]:
cont = Contractions(api_key="glove-twitter-100")
cont.load_models()


[=====---------------------------------------------] 11.9% 46.0/387.1MB downloaded

## Pre processing  Methods

In [7]:

#Version Edit: Sonam D.
def to_lower(text):
    text = text.lower()
    return text

#Version Edit: Anit G.
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+|pic.twitter.com\S+')
    return url.sub(r' ',text)

#Version Edit: Anit G.
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r' ',text)

#Version Edit: Anit G.
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', text)

#Version Edit: Sonam D.
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)


#Version Edit: Susanth D.
def remove_special_ucchar(text):
    text = re.sub('&.*?;', ' ', text)
    return text


#Version Edit: Sonam D.
def remove_numbers(text):
    text = re.sub(r'\d+', ' ', text)
    return text

#Version Edit: Sonam D.
def remove_mentions(text):
    text = re.sub(r'@\w*', ' ', text)
    return text

#Version Edit: Sonam D.
def remove_punctuations(text):
    text = re.sub(r'([^A-Za-z \t])|(\w+:\/\/\S+)', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_square_bracket(text):
    text = re.sub('\[.*?\]', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_angular_bracket(text):
    text = re.sub('\<.*?\>+', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_newline(text):
    text = re.sub('\n', ' ', text)
    return text

#Version Edit: Susanth D.
def remove_words_with_numbers(text):
    text = re.sub('\w*\d\w*', ' ', text)
    return text
    
#Version Edit: Susanth D.
def hashtag_to_words(text):
    text = re.sub(r'##', '#', text)
    hash_pattern = re.compile(r"#\w*")
    hashtag_list = re.findall(r"#\w+",text)
    for hashtag in hashtag_list:
        hashtag = re.sub(r'#', '', hashtag)
        text = re.sub(hashtag, sym_spell_4space.word_segmentation(hashtag).corrected_string, text)
    text = re.sub(r'#', '', text)
#     text = re.sub(r'# ', '', text)
    return text

#Version Edit: Susanth D.
def remove_stopwords(text):
    text_tokens=word_tokenize(text)
    textop = ''
    for token in text_tokens:
        if token not in stopwords.words('english'):
            textop = textop + token + ' '
    return textop

#Version Edit: Sonam D.
def correct_misspelled_with_context(text):
    suggestions = sym_spell_misspelled.lookup_compound(text, max_edit_distance=2)
    text = str(suggestions[0])
    text = re.sub(r', \d', ' ', text)
#     text = remove_numbers(text)
    return text

#Version Edit: Sonam D.
def stemming_text(text):
    stemmer= PorterStemmer()
    text_tokens=word_tokenize(text)
    textop = ''
    for token in text_tokens:
        textop = textop + stemmer.stem(token) + ' '
    return textop

#Version Edit: Sonam D.
def lemmatization(text):
    lemmatizer=WordNetLemmatizer()
    text_tokens=word_tokenize(text)
    textop = ''
    for token in text_tokens:
        textop = textop + lemmatizer.lemmatize(token) + ' '
    return textop

#Version Edit: Anit G.
def convert_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

#Version Edit: Anit G.
def convert_abbrev_in_text(text):
    tokens = word_tokenize(text)
    tokens = [convert_abbrev(word) for word in tokens]
    text = ' '.join(tokens)
    return text

#Version Edit: Sonam D.
# def handle_contractions(text):
#     text = re.sub(r"’", "'", text)
#     for word in text.split():
#         if word.lower() in contractions:
#             text = text.replace(word, contractions[word.lower()])
#     return(text)

#Version Edit: Saurabh M.
def removeRepeated(tweet):
    prev = ''
    tweet_new = ''
    for c in tweet:
        caps = False
        if c.isdigit():
            tweet_new += c
            continue
        if c.isalpha() == True:
            if ord(c) >= 65 and ord(c)<=90:
                caps = True
            c = c.lower()
            if c == prev:
                count += 1
            else:
                count = 1
                prev = c
            if count >= 3:
                continue
            if caps == True:
                tweet_new += c.upper()
            else:
                tweet_new += c
        else:
            tweet_new += c
    return tweet_new


def Expand_Contractions(text):
    return list(cont.expand_texts([text]))[0]

## Pre-processing Data Pipeline 


In [8]:
def count_chars(text):
    new_text = text.apply(lambda x : list(x)).explode()
    return new_text.unique().shape[0]

def count_words(text):
    new_text = text.apply(lambda x : x.split(' ')).explode()
    return new_text.unique().shape[0]

def preprocess_pipeline(steps, col, df):
    new_col = df[col]
    char_count_before = 0
    word_count_before = 0
    char_count_after = 0
    word_count_after = 0
    for each_step in steps:
        char_count_before = count_chars(new_col)
        word_count_before = count_words(new_col)
        new_col = new_col.apply(each_step)
        char_count_after = count_chars(new_col)
        word_count_after = count_words(new_col)
        print("Preprocessing step: ",each_step.__name__)
        print("Char Count ---> Before: %d | After: %d"%(char_count_before, char_count_after))
        print("Word Count ---> Before: %d | After: %d"%(word_count_before, word_count_after))
    
    return new_col
        
        

In [9]:
train = pd.read_csv('../input/nlp-getting-started/train.csv')
test = pd.read_csv('../input/nlp-getting-started/test.csv')


pipeline = []
# pipeline.append(to_lower)
pipeline.append(remove_newline)
pipeline.append(remove_URL)
pipeline.append(remove_html)
pipeline.append(remove_emoji)
pipeline.append(remove_special_ucchar)
# pipeline.append(hashtag_to_words)
# pipeline.append(remove_words_with_numbers)
# pipeline.append(remove_numbers)
pipeline.append(remove_mentions)
# pipeline.append(remove_square_bracket)
# pipeline.append(remove_angular_bracket)
pipeline.append(Expand_Contractions)
pipeline.append(remove_punctuations)
pipeline.append(removeRepeated)
# pipeline.append(convert_abbrev_in_text)
# pipeline.append(remove_stopwords)
# pipeline.append(correct_misspelled_with_context)
# pipeline.append(remove_numbers)
# pipeline.append(remove_stopwords)
# # pipeline.append(stemming_text)
pipeline.append(lemmatization)

train['processed_text'] = preprocess_pipeline(pipeline, 'text', train)
print()
test['processed_text'] = preprocess_pipeline(pipeline, 'text', test)
train.head()

Preprocessing step:  remove_newline
Char Count ---> Before: 122 | After: 121
Word Count ---> Before: 32017 | After: 31925
Preprocessing step:  remove_URL
Char Count ---> Before: 121 | After: 121
Word Count ---> Before: 31925 | After: 27333
Preprocessing step:  remove_html
Char Count ---> Before: 121 | After: 121
Word Count ---> Before: 27333 | After: 27333
Preprocessing step:  remove_emoji
Char Count ---> Before: 121 | After: 121
Word Count ---> Before: 27333 | After: 27333
Preprocessing step:  remove_special_ucchar
Char Count ---> Before: 121 | After: 120
Word Count ---> Before: 27333 | After: 27300
Preprocessing step:  remove_mentions
Char Count ---> Before: 120 | After: 119
Word Count ---> Before: 27300 | After: 24973
Preprocessing step:  Expand_Contractions
Char Count ---> Before: 119 | After: 119
Word Count ---> Before: 24973 | After: 24801
Preprocessing step:  remove_punctuations
Char Count ---> Before: 119 | After: 53
Word Count ---> Before: 24801 | After: 18130
Preprocessing st

,id,keyword,location,text,target,processed_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this earthquake Ma...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask Canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All resident asked to shelter in place are bei...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order in Ca...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby Alaska a sm...


In [10]:
u, idx = np.unique(train['processed_text'], return_index=True)
train.iloc[idx]

,id,keyword,location,text,target,processed_text
144,208,airplane%20accident,"Eagle Pass, Texas",A Cessna airplane accident in Ocampo Coahuila ...,1,A Cessna airplane accident in Ocampo Coahuila ...
478,688,attack,"Dayton, Ohio",A Dayton-area org tells me it was hit by a cyb...,1,A Dayton area org tell me it wa hit by a cyber...
7320,10479,wild%20fires,NaN,@EnzasBargains A5 Donated some fruit snacks &a...,1,A Donated some fruit snack handi wipe to our f...
7045,10092,typhoon,india,A GPM satellite 'bullseye' in Typhoon Soudelor...,1,A GPM satellite bullseye in Typhoon Soudelor
6858,9830,trauma,www.aprylpooley.com,A1: I started writing when I couldn't talk abo...,0,A I started writing when I could not talk abou...
...,...,...,...,...,...,...
1348,1949,burning%20buildings,NaN,@MoFanon ?? your last retweet you would think ...,0,your last retweet you would think the lion sav...
138,198,airplane%20accident,"Salt Lake City, Utah",@crobscarla your lifetime odds of dying from a...,0,your lifetime odds of dying from an airplane a...
3075,4411,electrocute,NaN,@i_electroCute your turn ??,0,your turn
6382,9120,suicide%20bomb,lagos. Unilag,16yr old PKK suicide bomber who detonated bomb...,1,yr old PKK suicide bomber who detonated bomb i...


In [11]:
tweet_len = train['processed_text'].apply(len)
print(tweet_len.max())
print(train['processed_text'].iloc[1270])

143
when you are taking a shower and someone flush the toilet and you have second to GTFO or you get burned 


In [12]:
tweet_len = test['processed_text'].apply(len)
print(tweet_len.max())

142


In [13]:
# pipeline=pdp.ApplyByCols('text',to_lower,'text')
# pipeline+=pdp.ApplyByCols('text',remove_newline,'text')
# pipeline+=pdp.ApplyByCols('text',remove_URL,'text')
# pipeline+=pdp.ApplyByCols('text',remove_html,'text')
# pipeline+=pdp.ApplyByCols('text',remove_emoji,'text')
# # pipeline+=pdp.ApplyByCols('text',hashtag_to_words,'text')
# pipeline+=pdp.ApplyByCols('text',remove_words_with_numbers,'text')
# # pipeline+=pdp.ApplyByCols('text',remove_numbers,'text')
# pipeline+=pdp.ApplyByCols('text',remove_mentions,'text')
# pipeline+=pdp.ApplyByCols('text',remove_square_bracket,'text')
# pipeline+=pdp.ApplyByCols('text',remove_angular_bracket,'text')
# pipeline+=pdp.ApplyByCols('text',Expand_Contractions,'text')
# pipeline+=pdp.ApplyByCols('text',remove_punctuations,'text')
# pipeline+=pdp.ApplyByCols('text',removeRepeated,'text')
# pipeline+=pdp.ApplyByCols('text',convert_abbrev_in_text,'text')
# pipeline+=pdp.ApplyByCols('text',correct_misspelled_with_context,'text')
# pipeline+=pdp.ApplyByCols('text',remove_numbers,'text')
# # pipeline+=pdp.ApplyByCols('text',remove_stopwords,'text')
# # pipeline+=pdp.ApplyByCols('text',stemming_text,'text')
# # pipeline+=pdp.ApplyByCols('text',lemmatization,'text')



# # df=pd.concat([train,test],sort=False)
# train=pipeline(train)
# test=pipeline(test)
# train.head()

In [14]:
# import xgboost as xgb
# import nltk
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# from sklearn.metrics import f1_score


In [15]:
# tokenizer5 = nltk.tokenize.TweetTokenizer()

# # appling tokenizer5
# train['text_tokens'] = train['text_processed'].apply(lambda x: tokenizer5.tokenize(x))
# train.head()

In [16]:
# xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, 
#                                                   train.target,
#                                                   random_state=42, 
#                                                   test_size=0.2, shuffle=True)
# print(xtrain.shape)
# print(xvalid.shape)

In [17]:
# # Appling CountVectorizer()
# count_vectorizer = CountVectorizer()
# xtrain_vectors = count_vectorizer.fit_transform(xtrain)
# xvalid_vectors = count_vectorizer.transform(xvalid)

In [18]:
# # Fitting a simple xgboost on CountVec
# clf = xgb.XGBClassifier(max_depth=10, n_estimators=300, colsample_bytree=0.3, 
#                         subsample=0.5, nthread=10, learning_rate=0.1, gamma=0.7, reg_alpha=0.01, reg_lambda=0.01)
# clf.fit(xtrain_vectors, ytrain)

# predictions = clf.predict(xvalid_vectors)
# print('XGBClassifier on CountVectorizer')
# print ("f1_score :", np.round(f1_score(yvalid, predictions),5))

In [19]:
# # Appling TFIDF
# tfidf = TfidfVectorizer(ngram_range=(1, 2), norm='l2')
# xtrain_tfidf = tfidf.fit_transform(xtrain)
# xvalid_tfidf = tfidf.transform(xvalid)

In [20]:
# # Fitting a simple xgboost on TFIDF
# clf = xgb.XGBClassifier(max_depth=100, n_estimators=300, colsample_bytree=0.3, 
#                         subsample=0.5, nthread=10, learning_rate=0.1)
# clf.fit(xtrain_tfidf.tocsc(), ytrain)
# predictions = clf.predict(xvalid_tfidf.tocsc())

# print('XGBClassifier on TFIDF')
# print ("f1_score :", np.round(f1_score(yvalid, predictions),5))

## Bert Model [Implementation 1]-  Methods

In [21]:
"""BERT Methods Predefined"""
def bert_encode(texts, tokenizer, max_len=150):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [22]:
print('There are {} rows and {} columns in train'.format(train.shape[0],train.shape[1]))
print('There are {} rows and {} columns in train'.format(test.shape[0],test.shape[1]))

There are 7613 rows and 6 columns in train
There are 3263 rows and 5 columns in train


# Bert Implementation

### Downloading a pretrained model

In [23]:
# Load BERT from the Tensorflow Hub

module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
# module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
#https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1
#https://tfhub.dev/tensorflow/bert_en_wwm_uncased_L-24_H-1024_A-16/1
bert_layer = hub.KerasLayer(module_url, trainable=True)




### **Tokenising Tweets**

In [24]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocab_file, do_lower_case)

In [25]:
# full_train_input = bert_encode(train.processed_text.values, tokenizer, max_len=160)
# test_input = bert_encode(test.processed_text.values, tokenizer, max_len=160)
# full_train_labels = train.target.values

In [26]:
train_data, val_data, train_labels, val_labels = train_test_split(train.processed_text.values, train.target.values, test_size=0.2)
train_input = bert_encode(train_data, tokenizer, max_len=150)
val_input = bert_encode(val_data, tokenizer, max_len=150)
test_input = bert_encode(test.processed_text.values, tokenizer, max_len=150)

In [27]:
from tensorflow.keras.optimizers import SGD
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras import regularizers
import keras.backend as K

In [28]:
learning_rate=9e-6
decay=1e-8
Dropout_num=0
max_len=150
# lr_schedule = [5e-8,1e-8,5e-9,1e-9]
# lr_schedule = [1e-6, 5e-7, 1e-7, 5e-8, 1e-8, 1e-9]
lr_schedule = [9e-7,1e-8,9e-8,5e-8,7e-9,1e-9]

In [29]:
input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
_, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
clf_output = sequence_output[:, 0, :]

## Type1
# out = Dense(100, activation='relu', activity_regularizer=regularizers.l2(9e-5))(clf_output)
# out = Dense(100, activation='relu', activity_regularizer=regularizers.l2(9e-5))(out)
# out = Dense(100, activation='relu')(out)

## Type2
out = Dense(1024, activation='relu', activity_regularizer=regularizers.l2(9e-5))(clf_output)

## Type3
# out = clf_output

out = Dense(1, activation='sigmoid')(out)

In [30]:
sBERT = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
# sBERT.compile(SGD(lr=learning_rate, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])
sBERT.compile(Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
sBERT.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 150)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 150)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 1024), (None 335141889   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [31]:
early_stopping = EarlyStopping(monitor='val_accuracy',
                               min_delta=0,
                               patience=2,
                               verbose=1, 
                               mode='auto')

checkpoint1 = ModelCheckpoint('best_accuracy.h5',
                             monitor='val_accuracy',
                             save_best_only=True)
# checkpoint2 = ModelCheckpoint('best_loss.h5',
#                              monitor='val_loss',
#                              save_best_only=True)


train_history = sBERT.fit(
    train_input, train_labels,
    validation_data=(val_input, val_labels),
    epochs = 1,
    callbacks=[checkpoint1],
    batch_size = 16
)

Train on 6090 samples, validate on 1523 samples
6090/6090 [==============================] - 437s 72ms/sample - loss: 0.4296 - accuracy: 0.8184 - val_loss: 0.4072 - val_accuracy: 0.8260


In [32]:
# K.set_value(sBERT.optimizer.lr, 1e-6)
# idx = np.random.permutation(train_input[0].shape[0])
# input1, input2, input3 = train_input
# input1 =input1[idx]
# input2 =input2[idx]
# input3 =input3[idx]
# train_input = input1, input2, input3
# train_labels = train_labels[idx]
# train_history = sBERT.fit(
#     train_input, train_labels,
#     validation_data = (val_input, val_labels),
#     epochs = 1, # recomended 3-5 epochs
#     callbacks=[checkpoint1, checkpoint2],
#     batch_size = 48 # recomended 8-16
# )

In [33]:
for i in range(5):
    K.set_value(sBERT.optimizer.lr, lr_schedule[i])
    idx = np.random.permutation(train_input[0].shape[0])
    input1, input2, input3 = train_input
    input1 = input1[idx]
    input2 = input2[idx]
    input3 = input3[idx]
    train_input = input1, input2, input3
    train_labels = train_labels[idx]
    train_history = sBERT.fit(
        train_input, train_labels,
        validation_data = (val_input, val_labels),
        epochs = 1, 
        steps_per_epoch = 200,
        callbacks = [checkpoint1],
        batch_size = 16 
    )


Train on 6090 samples, validate on 1523 samples
3200/6090 [==============>...............] - ETA: 3:37 - loss: 0.2434 - accuracy: 0.9097 - val_loss: 0.4207 - val_accuracy: 0.8293

In [34]:
# train_history = sBERT.fit(
#     train_input, train_labels,
#     validation_split = 0.2,
#     epochs = 1, # recomended 3-5 epochs
# #     steps_per_epoch=320,
#     callbacks=[checkpoint1],
#     batch_size = 16 # recomended 8-16
# )

In [35]:
sub = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

In [36]:
# Susanth's Full model
test_pred = sBERT.predict(test_input)
sub['target'] = test_pred.round().astype(int)
sub.to_csv('submission full.csv', index=False)
sub.head(3)

,id,target
0,0,1
1,2,1
2,3,1


In [37]:
## Susanth's checkpoint model
sBERT.load_weights('best_accuracy.h5')
test_pred = sBERT.predict(test_input)
sub['target'] = test_pred.round().astype(int)
sub.to_csv('submission checkpoint.csv', index=False)
sub.head(3)


,id,target
0,0,1
1,2,1
2,3,1


In [38]:
# ## Susanth's checkpoint model
# sBERT.load_weights('best_loss.h5')
# test_pred = sBERT.predict(test_input)
# sub['target'] = test_pred.round().astype(int)
# sub.to_csv('submission checkpoint best loss.csv', index=False)
# sub.head(3)

## Tuning Parameters Tried and Respective Accuracies for BERT

In [39]:
################## Tune Parameter 1 ##################
# Dropout_num = 0
# learning_rate = 1e-7
# valid = 0.3
# epochs_num = 10
# batch_size_num = 32
# Dimensions = 768
# Dense Layer = 1(Final)
# Val_accuracy : 79.9
# Test_accuracy : 90.31
######################################################
################## Tune Parameter 2 ##################
# Dropout_num = 0
# learning_rate = 1e-7
# valid = 0.3
# epochs_num = 10
# batch_size_num = 32
# Dimensions = 768
# Dense Layer = 1
# Dense 1 = 768 (Relu)
# Desne 2 = 1 (Softmax)
# Val_accuracy : 80.7
# Test_accuracy : 90.20
######################################################
################## Tune Parameter 3 ##################
#Dropout_num = 0
#learning_rate = 9e-6
#valid = 0.1
#epochs_num = 3
#batch_size_num = 16
#Val_accuracy : 
######################################################
################## Tune Parameter 4 ##################
#Dropout_num = 0
#learning_rate = 5e-6
#valid = 0.1
#epochs_num = 3
#batch_size_num = 16
#Val_accuracy : 81.63
######################################################
################## Tune Parameter 5 ##################
#Dropout_num = 0
#learning_rate = 5e-6
#valid = 0.1
#epochs_num = 5
#batch_size_num = 16
#Val_accuracy : 
######################################################
################## Tune Parameter 6 ##################
#Dropout_num = 0
#learning_rate = 9e-6
#valid = 0.1
#epochs_num = 5
#batch_size_num = 16
#Val_accuracy : 
######################################################